# Benchmark Performance of argmin()

In [1]:
using BenchmarkTools

In [2]:
x = rand(450);

In [16]:
@btime argmin(x)

  2.032 μs (0 allocations: 0 bytes)


41

In [18]:
@btime argmin(@view x[1:449])

  530.700 ns (1 allocation: 48 bytes)


41

In [19]:
@btime argmin(@view x[1:448])

  528.068 ns (1 allocation: 48 bytes)


41

In [6]:
@benchmark argmin(x)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.032 μs …   8.125 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.046 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.114 μs ± 238.660 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄  ▄▁ ▁                                                    ▁
  ██▅▆██▇█▇▇▆▆▆▆▇▆▆▇▇▆▇▇▇█▆▆▆▇▆▆▅▇▅▃▃▅▅▄▅▅▃▄▅▄▄▃▄▅▃▅▄▂▃▃▄▄▄▃▄ █
  2.03 μs      Histogram: log(frequency) by time       3.3 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
using LoopVectorization

In [8]:
fast_argmin(dij, n) = begin
    best = 1
    @inbounds dij_min = dij[1]
    @turbo for here in 2:n
        newmin = dij[here] < dij_min
        best = newmin ? here : best
        dij_min = newmin ? dij[here] : dij_min
    end
    dij_min, best
end

fast_argmin (generic function with 1 method)

In [9]:
@btime fast_argmin(x, 450)

  97.207 ns (1 allocation: 32 bytes)


(0.000130064906763816, 41)

In [10]:
@benchmark fast_argmin(x, 450)

BenchmarkTools.Trial: 10000 samples with 946 evaluations.
 Range (min … max):   96.634 ns …  1.384 μs  ┊ GC (min … max): 0.00% … 90.72%
 Time  (median):      97.119 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   100.290 ns ± 28.165 ns  ┊ GC (mean ± σ):  0.67% ±  2.22%

  ██▃   ▁ ▂▂▁▃▄▃▂▂▂▁▁▂▁▁▁▂▂▁ ▂▂▁▁ ▁▁▁▁                         ▂
  █████████████████████████████████████▆██▇▇▇▇▇█▇▇▆▇▆▆▇▆▇▆▅▆▆▆ █
  96.6 ns       Histogram: log(frequency) by time       116 ns <

 Memory estimate: 32 bytes, allocs estimate: 1.

In [11]:
@benchmark for j in 450:-1:1 fast_argmin(x, j) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  25.916 μs …  1.364 ms  ┊ GC (min … max): 0.00% … 96.64%
 Time  (median):     26.792 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   28.052 μs ± 22.371 μs  ┊ GC (mean ± σ):  1.35% ±  1.67%

  ▆███▆▆▆▆▅▄▃▃▃▃▂▂▁▁▁▂▂▃▃▂▂▂▁▂ ▁                              ▂
  ████████████████████████████████▇▇█████▇█▇▇▆▆▅▆▄▆▅▄▄▂▄▅▃▅▄▅ █
  25.9 μs      Histogram: log(frequency) by time        38 μs <

 Memory estimate: 14.06 KiB, allocs estimate: 450.

In [12]:
@benchmark for j in 450:-1:1 argmin(@view x[1:j]) end

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  123.083 μs …  1.885 ms  ┊ GC (min … max): 0.00% … 92.43%
 Time  (median):     124.334 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   128.472 μs ± 45.709 μs  ┊ GC (mean ± σ):  0.92% ±  2.43%

  █▅▇▄▃▂▂▂▄▄▄▄▃▃▂▂▂▂▂▃▂▂▂▁▂▁▁▁▁ ▁▁▁ ▁                          ▂
  ███████████████████████████████████████▇▇▇▇▇▇▇▆▆▇▆▇▄▆▆▅▅▄▆▄▄ █
  123 μs        Histogram: log(frequency) by time       147 μs <

 Memory estimate: 35.16 KiB, allocs estimate: 900.